In [20]:
import nltk, re, pprint
from nltk import word_tokenize
import os,json
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
import nltk.data
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize
import augur
import sqlalchemy as s
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import table, column, select, update, insert
from sqlalchemy import MetaData
import pandas as pd
%matplotlib inline
#nltk.download('punkt')
#nltk.download('stopwords')
#pip install twython
#nltk.download('vader_lexicon')

In [21]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect = augurApp.ghtorrentplus()

In [22]:
table_names = s.inspect(connect.db).get_table_names()
print(table_names)

['issue_response_time', 'mail_lists', 'mailing_list_jobs', 'sentiment_scores']


In [4]:
if("mailing_list_jobs" in table_names):
    lists_createdSQL = s.sql.text("""SELECT project FROM mailing_list_jobs""")
    df1 = pd.read_sql(lists_createdSQL, connect.db)
    print(df1)
    val = True

          project
0      aalldp-dev
1  archetypes-dev
2        announce


In [5]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sid = SentimentIntensityAnalyzer()
col = 'score','sentiment'
df3 = pd.DataFrame(columns = col)
for i in df1['project'].values:
    print(i,"Mailing List")
    SQL = s.sql.text("""SELECT subject,message_id,message_text,message_parts_tot 
    FROM mail_lists WHERE mailing_list = """ +  "'" + i + "'")
    df2 = pd.read_sql(SQL, connect.db)
    #print(df2)
    grouped = df2.groupby('message_id').groups
    print("Here!!!!")
    #print("\n\n","-"*70,"\n\n")
    for group in grouped:
        parts = 0
        numb = len(df2.loc[df2['message_id'] == group]['message_parts_tot'].tolist())
        message = (df2.loc[df2['message_id'] == group]['message_text']).tolist()
        message_text = ''.join(message)
        sentences = tokenizer.tokenize(message_text)
        compound = parts = 0
        sentiment = "Positive"
        for sentence in sentences:
            scores = sid.polarity_scores(sentence)
            compound+= scores['compound']
            parts+=1
        avg_score = compound/parts
        if(avg_score == 0):
            sentiment = "Neutral"
        elif(avg_score < 0):
            sentiment = "Negative"
        #print(message)
        print("\n\n")
        #print("Score",avg_score)
        print("\n\n\n")
        for i in range(numb):
            li = [ [avg_score, sentiment] ]
            df_temp = pd.DataFrame(li,columns = col)
            #print(df_temp['score'])
            df3 = df3.append(df_temp)
            #print(df3)
    print("\n\n")

aalldp-dev Mailing List
Here!!!!

















archetypes-dev Mailing List
Here!!!!

















announce Mailing List
Here!!!!




































































In [8]:
if(len(list(df3)) == 2):
    df3.reset_index(level=0, inplace=True)
    df3.columns = ['augurmsgID','score','sentiment']
    for i in range(len(df3['augurmsgID'])):
        df3.loc[i,'augurmsgID'] = i+1
print(df3)

#for i in range(len(df4['score'])):

    augurmsgID     score sentiment
0            1  0.140263  Positive
1            2  0.140263  Positive
2            3  0.624650  Positive
3            4  0.624650  Positive
4            5  0.148000  Positive
5            6  0.148000  Positive
6            7  0.285950  Positive
7            8  0.285950  Positive
8            9  0.254578  Positive
9           10  0.254578  Positive
10          11  0.405283  Positive
11          12  0.405283  Positive
12          13  0.172427  Positive
13          14  0.172427  Positive
14          15  0.264620  Positive
15          16  0.264620  Positive
16          17  0.350940  Positive
17          18  0.350940  Positive
18          19  0.191714  Positive
19          20  0.191714  Positive
20          21  0.170033  Positive
21          22  0.170033  Positive
22          23  0.344075  Positive
23          24  0.344075  Positive
24          25  0.481638  Positive
25          26  0.481638  Positive


In [10]:
df3.to_sql(name='sentiment_scores',con=connect.db,if_exists='replace',index=False)

In [23]:
Session = sessionmaker(connect.db)
session = Session()

In [42]:
columns1 = 'augurmsgID','score','sentiment'

df6 = pd.DataFrame(columns=columns1)
df6.to_sql(name="table_new", con=connect.db,if_exists='replace',index=False,
					dtype={'augurmsgID': s.types.Integer,
                        'score':s.types.Float,
                        'sentiment':s.types.VARCHAR(length=15)
					})

In [44]:
session.execute("""SELECT

 mail_lists.augurmsgID as augurmsgID, 
 sentiment_scores.score as score,
 sentiment_scores.sentiment as sentiment

INTO table_new

FROM mail_lists

JOIN sentiment_scores ON mail_lists.augurmsgID 
= sentiment_scores.augurmsgID;""")

InternalError: (pymysql.err.InternalError) (1327, 'Undeclared variable: table_new') [SQL: 'SELECT\n\n mail_lists.augurmsgID as augurmsgID, \n sentiment_scores.score as score,\n sentiment_scores.sentiment as sentiment\n\nINTO table_new\n\nFROM mail_lists\n\nJOIN sentiment_scores ON mail_lists.augurmsgID \n= sentiment_scores.augurmsgID;'] (Background on this error at: http://sqlalche.me/e/2j85)